# Quiz- Implementing Harris Corner Detector

The Harris corner detection algorithm also called the Harris & Stephens corner detector is one of the simplest corner detectors available. The idea is to locate interest points where the surrounding neighbourhood shows edges in more than one direction.The algorithm is to find the difference in intensity for a displacement of (u,v) in all directions.
Follow the steps in the given in this notebook to implement Harris corner detector using numpy


The first step is to calculte image gadients in both horizontal and Vertical direction. We are going to use convolution operation that you learnt in second week to find gradients of the image. In case you are not familiar with convolution operation much, Watch this playlist: https://www.youtube.com/playlist?list=PLskqk7JOmYpxwGkgbuhgIS9Hqg0FjQkD1. It will be very usefull.

In [1]:
#importing packages
import cv2
import numpy as np

### 3.1 - Zero-Padding

Zero-padding adds zeros around the border of an image:

<img src="images/PAD.png" style="width:600px;height:400px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **Zero-Padding**<br> Image (3 channels, RGB) with a padding of 2. </center></caption>

The main benefits of padding are the following:

- It helps us keep more of the information at the border of an image. Without padding, very few values at the next layer would be affected by pixels as the edges of an image.

**Exercise**: Implement the following function, which pads image with zeros. [Use np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html). Note if you want to pad the array "a" of shape $(5,5,5,5,5)$ with `pad = 1` for the 2nd dimension, `pad = 3` for the 4th dimension and `pad = 0` for the rest, you would do:
```python
a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode='constant', constant_values = (0,0))
```

In [1]:
def zero_pad(img, pad):
    '''
    Argument:
    img -- input image
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    Returns:
    img_pad -- padded image of shape ( n_H + 2*pad, n_W + 2*pad)
    '''
    ### START CODE HERE ### (≈ 1 line)
    img_pad = None
    ### END CODE HERE ###
    return img_pad

### 3.2 - Single step of convolution 

In this part, implement a single step of convolution, in which you apply the filter to a single position of the input. This will be used to build a convolutional unit, which: 

- Takes an input image 
- Applies a filter at every position of the input
- Outputs convolved image (usually of different size)

<img src="images/Convolution_schematic.gif" style="width:500px;height:300px;">
<caption><center> <u> <font color='purple'> **Figure 2** </u><font color='purple'>  : **Convolution operation**<br> with a filter of 3x3 and a stride of 1 (stride = amount you move the window each time you slide) </center></caption>

In a computer vision application, each value in the matrix on the left corresponds to a single pixel value, and we convolve a 3x3 filter with the image by multiplying its values element-wise with the original matrix, then summing them up. In this first step of the exercise, you will implement a single step of convolution, corresponding to applying a filter to just one of the positions to get a single real-valued output. 

Later in this notebook, you'll apply this function to multiple positions of the input to implement the full convolutional operation. 

**Exercise**: Implement conv_single_step(). [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html).

In [2]:
def conv_single_step(img,kernel):
    
    ### START CODE HERE ### (≈ 2 lines of code)
    # Element-wise product between img and kernel
    s = None
    # Sum over all entries of the s.
    Z = None
    ### END CODE HERE ###
    return Z
    


**Reminder**:
The formulas relating the output shape of the convolution to the input shape is:
$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$


#### Additional Hints if you're stuck


* You will want to use array slicing (e.g.`varname[0:1,:,3:5]`) for the following variables:  
  `img_pad` ,`kernel`.  
  Copy the starter code of the function and run it outside of the defined function, in separate cells.  
  Check that the subset of each array is the size and dimension that you're expecting.  
* To decide how to get the vert_start, vert_end; horiz_start, horiz_end, remember that these are indices of the previous layer.  
  Draw an example of a previous padded layer (8 x 8, for instance), and the current (output layer) (2 x 2, for instance).  
  The output layer's indices are denoted by `h` and `w`.  

In [3]:
def conv_forward(img, Kernel,pad):
    ### START CODE HERE ###
    # Retrieve dimensions from img's shape (≈1 line)  
    (n_H_prev, n_W_prev) = None
    
    # Retrieve dimensions from kernel's shape (≈1 line)
    (f, f) = None
    
    # Compute the dimensions of the CONV output using the formula given above. 
    # Hint: use int() to apply the 'floor' operation. (≈2 lines)
    n_H = None
    n_W = None
    
    # Initialize the output volume Z with zeros. (≈1 line)
    Z = np.zeros((n_H,n_W))
    
    # Create a padded image using function you wrote above. choose pad such that i/p image dimention is equal to the output.
    img_pad = zero_pad(img,pad)
    

    for h in range(n_H_prev):           # loop over vertical axis of the output volume
        # Find the vertical start and end of the current "slice"(≈2 lines)
        vert_start = None
        vert_end = None

        for w in range(n_W_prev):       # loop over horizontal axis of the output volume
            # Find the horizontal start and end of the current "slice" (≈2 lines)
            horiz_start = None
            horiz_end = None

            # Use the corners to define the slice of image (See Hint above the cell). 
            img_slice= None
            Z[ h, w] = None
    ### END CODE HERE ###

    # Making sure your output shape is correct
    assert(Z.shape == (n_H, n_W))

    return Z


As mentioned in the note book compute the structre tensor by the following equation:
        $A=\sum_{u} \sum_{v} w(u, v)\left[\begin{array}{cc}I_{x}^{2} & I_{x} I_{y} \\ I_{x} I_{y} & I_{y}^{2}\end{array}\right]=\left[\begin{array}{cc}\left\langle I_{x}^{2}\right\rangle & \left\langle I_{x} I_{y}\right\rangle \\ \left\langle I_{x} I_{y}\right\rangle & \left\langle I_{y}^{2}\right\rangle\end{array}\right]$
        


In [4]:
def A(Ix,Iy,win_size,height,width):
    #Ix and Iy are partial derivates ofd the funtion in x and y diectyon respectively
    Ixx= None
    #take element wise product Ix*Ix
    Iyy= None  
    #take element wise product Iy*Iy
    Ixy= None  
    #take element wise product Ix*Iy
    i=int(win_size/2)
    #Calculate A by windowing the above matrices and summing
    #Take a look at this. This has been implemented for you :)
    S=np.zeros((height-i,width-i,2,2))
    for y in range(i,height-i):
        for x in range(i,width-i):
            windowxx=Ixx[y-i:y+i+1,x-i:x+i+1]
            windowyy=Iyy[y-i:y+i+1,x-i:x+i+1]
            windowxy=Ixy[y-i:y+i+1,x-i:x+i+1]
            Sxx=windowxx.sum()
            Syy=windowyy.sum()
            Sxy=windowxy.sum()
            S[y,x]=[[Sxx,Sxy],[Sxy,Syy]]
    # So the matrix A is [[Sxx,Sxy],[Sxy,Syy]]
    return S

compute the determinat:
$M_{c}=\lambda_{1} \lambda_{2}-\kappa\left(\lambda_{1}+\lambda_{2}\right)^{2}=\operatorname{det}(A)-\kappa \operatorname{trace}^{2}(A)$


In [5]:
def resultant(S,k):
    #create an array of zeros with dimention of(S.shape[0],S.shape[1])
    res=None
    for y in range(S.shape[0]):
        for x in range(S.shape[1]):
            #compute determinat
            det=None
            trace=None
            
            #compute function
            function=None
            res[y,x]=None
            
    return res
            
    

Integrate all the above functions we discussed and implement harris corner detector.use kernels [[1,2,1],[0,0,0],[-1,-2,-1]] and [[1,0,-1],[2,0,-2],[1,0,-1]]

In [6]:
img1=cv2.imread('..\\images\\cc.png')
img=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY )
copy=img1.copy()
K1=None
K2=None
Ix=None
Iy=None
S=None
k=0.05
res=None
threshold=100000
for y in range(s.shape[0]):
    for x in range(s.shape[1]):
        if res[y,x]>threshold:
            copy[y,x]=[0,0,255]
cv2.imshow('detected',copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined